In [1]:
import os
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]


llm = ChatOllama(model="gemma3:1b")

In [3]:
def process(state: AgentState) -> AgentState:
    """This node will solve the request you input"""
    response = llm.invoke(state["messages"])

    state["messages"].append(AIMessage(content=response.content))
    print(f"\nAI: {response.content}")
    print("CURRENT STATE: ", state["messages"])

    return state

In [4]:
graph = StateGraph(AgentState)

graph.add_node("process", process)

graph.add_edge(START, "process")
graph.add_edge("process", END)

agent = graph.compile()

In [5]:
conversation_history = []

user_input = input("Enter: ")

while user_input != "exit":
    conversation_history.append(HumanMessage(content=user_input))

    result = agent.invoke({"messages": conversation_history})

    conversation_history = result["messages"]

    user_input = input("Enter: ")


AI: Hi there! How's your day going? 😊 

Is there anything you'd like to chat about or need help with?
CURRENT STATE:  [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi there! How's your day going? 😊 \n\nIs there anything you'd like to chat about or need help with?", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

AI: Hi Yash! It’s great to meet you. 😊 

It's nice to hear from you. What’s on your mind today?
CURRENT STATE:  [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi there! How's your day going? 😊 \n\nIs there anything you'd like to chat about or need help with?", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='my name is yash', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Yash! It’s great to meet you. 😊 \n\nIt's nice to hear from you. What’s on your mind today?", additional_kwa

In [6]:
with open("logging.txt", "w", encoding="utf-8") as file:
    file.write("Your Conversation Log:\n")

    for message in conversation_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
    file.write("End of Conversation")

print("Conversation saved to logging.txt")

Conversation saved to logging.txt
